In [ ]:
import pandas as pd
import numpy as np
import scrapy
from scrapy import Selector
from scrapy.http import HtmlResponse
import time
import requests
import certifi
import urllib3
urllib3.disable_warnings()

In [ ]:
#Trích xuất dữ liệu từ nguồn html của trang web
url = "https://dmec.moh.gov.vn/van-ban-cong-bo?p_p_id=vanbancongbo_WAR_trangthietbiyteportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_vanbancongbo_WAR_trangthietbiyteportlet_keyword=&_vanbancongbo_WAR_trangthietbiyteportlet_showHide=0&_vanbancongbo_WAR_trangthietbiyteportlet_ngayGuiTu=&_vanbancongbo_WAR_trangthietbiyteportlet_ngayGuiDen=&_vanbancongbo_WAR_trangthietbiyteportlet_ngayCongBoTu=&_vanbancongbo_WAR_trangthietbiyteportlet_ngayCongBoDen=&_vanbancongbo_WAR_trangthietbiyteportlet_tenDoanhNghiep=&_vanbancongbo_WAR_trangthietbiyteportlet_tenTTBYT=&_vanbancongbo_WAR_trangthietbiyteportlet_coQuanQuanLyId=0&_vanbancongbo_WAR_trangthietbiyteportlet_tthcId=0&_vanbancongbo_WAR_trangthietbiyteportlet_keywords=&_vanbancongbo_WAR_trangthietbiyteportlet_advancedSearch=false&_vanbancongbo_WAR_trangthietbiyteportlet_andOperator=true&_vanbancongbo_WAR_trangthietbiyteportlet_resetCur=false&_vanbancongbo_WAR_trangthietbiyteportlet_delta=50"
html = requests.get(url,verify=False).content
sel = Selector(text=html)
table = sel.xpath('//*[@class="oep-table"]//tr')
ngay_cong_bo=[]
so_van_ban=[]
van_ban=[]
company_name=[]
mst=[]
dia_chi=[]
phuong=[]
quan=[]
thanh_pho=[]
phone=[]
email=[]
nguoi_dai_dien=[]
phone_nguoi_dai_dien=[]
tbyt_type=[]
x = range(1,51)
#Chọn số trang muốn crawl (ví dụ ở đây crawl 3 pages)
y = range(1,3)
for t in y:
    for i in x:
        so_van_ban.append(table[i].xpath('.//td[3]').xpath('.//text()').extract_first())
        ngay_cong_bo.append(table[i].xpath('.//td[4]').xpath('.//label[@class="date"]//text()').extract_first().split(": ")[1].split(" \xa0")[0])
        van_ban.append(table[i].xpath('.//a//text()').extract_first())
        file_links = table[i].xpath('.//a//@href')[0]
        links_to_follow = file_links.extract()
        html2=requests.get(links_to_follow,verify=False).content
        sel2 = Selector( text= html2 )
        
        company_name.append(sel2.xpath('//span[@class="text_value"]')[6].xpath('./text()').extract_first())
        mst.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]//span[@class="text_value"][1]/text()').extract_first())
        dia_chi.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]/p[2]/span[@class="text_value"]/text()').extract_first().split(", \n\t\t\t\t\t")[0])
        phuong.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]/p[2]/span[@class="text_value"]/text()').extract_first().split(", \n\t\t\t\t\t")[1])
        quan.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]/p[2]/span[@class="text_value"]/text()').extract_first().split(", \n\t\t\t\t\t")[2])
        thanh_pho.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]/p[2]/span[@class="text_value"]/text()').extract_first().split(", \n\t\t\t\t\t")[3])
        
        if len(sel2.xpath('//div[@style = "padding-left:15px;"][1]//span[@class="text_value"]'))> 5: 
            phone.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]//span[@class="text_value"][3]/text()').extract_first())
            email.append(sel2.xpath('//div[@style = "padding-left:15px;"][1]//span[@class="text_value"][6]/text()').extract_first())
        else:
            phone.append('')
            email.append('')
        nguoi_dai_dien.append(sel2.xpath('//div[@style = "padding-left:15px;"][2]/p[1]/span[@class="text_value"]').xpath('./text()').extract_first())
        phone_nguoi_dai_dien.append(sel2.xpath('//div[@style = "padding-left:15px;"][2]/p[3]/span[@class="text_value"]').xpath('./text()').extract_first())
        if sel2.xpath('//div[@style = "margin:15px 0;"]/p[4]').xpath('./text()').extract_first() == "4. Danh mục trang thiết bị y tế do cơ sở thực hiện mua bán:":
            tbyt_type.append(sel2.xpath('//div[@class = "main_don"]')[0].xpath('.//td[@class="text-left"]')[-1].xpath('./text()').extract_first())
        else:  
            tbyt_type.append(sel2.xpath('//div[@style = "padding-left:15px;"]')[2].xpath('.//span[@class="text_value"]')[6].xpath('./text()').extract_first())
        #tbyt_type.append(sel2.xpath('//div[@style = "padding-left:15px;"]')[2].xpath('.//span[@class="text_value"]')[6].xpath('./text()').extract_first())
        
    next_page = sel.xpath('//ul[@class = "pager lfr-pagination-buttons"]//a/@href')[2].extract()
    html = requests.get(next_page,verify=False).content
    sel = Selector( text= html )
    table = sel.xpath('//*[@class="oep-table"]//tr')

In [ ]:
#export data dưới dạng Excel
data = {
    'Ngay cong bo': ngay_cong_bo,
    'So van ban': so_van_ban,
    'Ten van ban': van_ban,
    'Ten cong ty': company_name,
    'Ma so thue': mst,
    'Dia chi chi tiet': dia_chi,
    'Phuong': phuong,
    'Quan': quan,
    'Thanh pho': thanh_pho,
    'Dien thoai': phone,
    'Email': email,
    'Nguoi dai dien': nguoi_dai_dien,
    'Dien thoai nguoi dai dien': phone_nguoi_dai_dien,
    'Loai TBYT': tbyt_type,
}
df = pd.DataFrame(data)
#print(df)
df.to_excel('Raw_KH_TBYT.xlsx', index=False)